### Connectie met Digital Human

by Kyra Jongman

### Inhoudsopgave

- Importeren packages
- Code voor het voeren van een gesprek

### Importeren packages

In [ ]:
import numpy as np
from flask import Flask, request, make_response
import json
import pickle
from flask_cors import cross_origin
from simplet5 import SimpleT5
from datetime import datetime, timedelta
import time

### Code voor het voeren van een gesprek

Met behulp van de onderstaande code wordt  via Ngrok een connectie met DialogFlow gemaakt. Deze code moet gerund worden wanneer een gesprek met de digital human aangegaan wordt. 

In [15]:
app = Flask(__name__)

model = pickle.load(open('t5model.pkl', 'rb'))

@app.route('/webhook', methods=['POST'])
def webhook():
    req = request.get_json(silent=True, force=True)
    res = processRequest(req)
    res = json.dumps(res, indent=4)
    
    r = make_response(res)
    r.headers['Content-Type'] = 'application/json'
    return r

class Conversation:

    def __init__(self):
        self.context = '<C>'
        self.annotation_list = []

    def user_sentence(self, user_sentence):
        user_sentence = "<U>" + user_sentence
        if 'Okay' in user_sentence or 'Great' in user_sentence:
            a = []
            for i in self.annotation_list:
                #attribute names
                try:
                    a.append(i.split("PAV>")[0].split(".")[1] + " " + i.split("PAV>")[0].split(".")[2] + " is")
                except:
                    try: 
                        a.append(i.split("PAV>")[0].split(".")[1].split("_")[0] + " " + i.split("PAV>")[0].split(".")[1].split("_")[1] + " is")
                    except:
                        a.append(i.split("PAV>")[0].split(".")[1] + " is")
                #attribute values
                a.append(i.split("PAV>")[1])
            a.insert(0, "Finding a suitable hotel for the customer with the help of the following attributes:")
            test = ' '.join(a) 
            return test
        else:
            answer = model.predict(user_sentence + self.context)[0]
            if 'PAN>' in answer:
                if 'A>' not in answer:
                    answer += " A>" + answer.split("PAV>")[1] +'?'
                    self.annotation_list.extend([answer.split(" A>")[0]])
                else:
                    self.annotation_list.extend([answer.split(" A>")[0]])
            new_context = user_sentence + " " + "<" + answer
            self.context = self.context + " " + new_context
            return answer.split("A>")[1]

conv = Conversation()    
def processRequest(req):  
    text = req.get("queryText")
    result = req.get("queryResult")
    parameters = result.get("parameters")
    user_question = parameters.get("userInput")
    
    intent = result.get("intent").get('displayName')
    if (intent=='Default Welcome Intent'):
        prediction = conv.user_sentence(user_question)     
        fulfillmentText = prediction 
        return {"fulfillmentText": fulfillmentText}
    elif (intent=='prediction'):
        prediction = conv.user_sentence(user_question)      
        fulfillmentText = prediction 
        return {"fulfillmentText": fulfillmentText}

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/May/2022 16:48:23] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:48:28] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:48:36] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:48:42] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:48:49] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:48:59] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:49:05] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:49:09] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:49:15] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2022 16:49:19] "POST /webhook HTTP/1.1" 200 -
